In [ ]:
# Django 프로젝트 생성
django-admin startproject mysite
## 터미널에서 입력

### polls 애플리케이션 디렉토리 및 하위 필요 파일 생성.
1. 터미널 해당 폴더 이동
2. 명령어 입력
> python manage.py startapp polls

### 프로젝트 설정 파일 변경
1. 터미덜 해당폴더 이동
2. 명령어 입력 (노트패드 open)
> notepad settings.py

* 설정 내용
1. ALLOWED_HOSTS 항목 적절하게 지정
```
DEBUG = True : 개발모드
        False : 운영모드
        * 운영모드일떈 반드시 서버, IP의 도메인 지정해야함.
        * 개발모드일 경우 default
        ['localhost','127.0.0.1']
* 필자처럼 리눅스 192.168.56.101 서버에서 개발시
ALLOWED_HOSTS = ['192.168.56.101','localhost','127.0.0.1']
```
2.  프로젝트에 포함되는 앱들 모두 설정파일에 등록해야함
(polls 도 포함)
```
INSTALLED_APPS = [
    'pools.app.PollsConfig',    # 추가
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
```
장고가 설정 클래스 찾을 수 있도록 모듈 경로까지 포함하여 등록
'polls.apps.PollsConfig'

3. 프로젝트에 사용할 DB 엔진 설정.
```
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

```
4. 타임존 지정 (세계 표준시 => 한국시간으로)
```
# TIME_ZONE = 'UTC'
TIME_ZONE = 'Asia/Seoul'
```
* USE_TZ = True 라고 설정 시, 장고가 알아서 시간대 조정함.
일광절약시간제 : Daylight Saving Time
=> 설정
: USE_TZ = True
    - 데이터베이스에 저장되는 시간은 UTC인데 Local은 한국 시간대일때.

* 근데, 둘 다 한국시간대면 그냥
: USE_TZ = False
상세 내용
-> https://docs.djangoproject.com/en/4.0/topics/il8n/timezones/

### 기본 테이블 생성
* migrate 명령 : DB에 변경사항 있을 시 이를 반영해주는 명령어
> python manage.py migrate
* 장고 사용 시, 프로그래밍 전에 이미 테이블 만들어야함
  - 기본적으로, 프로그래밍 시 반드시  사용자와 그룹 테이블 등이 필요하다는 가정 하에 만들어져있기 때문.